# 5291 Project

## Import data

In [0]:
#authorization from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/ADA-Final-Project/"

'5291 Project matrix factorization.ipynb'   Model.gdoc
'91Project - Group 6.pdf'		    Presentation.gdoc
 ADA_ContentBased.ipynb			    Santander_Product_Recommender_EDA
 ADA_DataCleaning.ipynb			    Step0_DataClearning.ipynb
'Copy of XGBoost.ipynb'			    test_1605.csv
 data_clean_v2.csv			    train_clean.csv
 data_clean_v3.csv			    train.csv
 Data.gdoc				    Untitled0.ipynb
 EDA.gdoc				    XGBoost.ipynb
 EDA_graph				    x_test_1605.csv
'Final Presentation.gdoc'		    x_train.csv
'Final Report.gdoc'			    彤狗连写好的code也跑不出来？.ipynb
 Group6.ipynb


### Install packages

In [0]:
!pip install --upgrade surprise

     |████████████████████████████████| 6.5MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678236 sha256=d356dead5b864ce0ae1db1c6f877ac79b66f5829efbb591d7af471b5b6211a0f
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from surprise import SVDpp
from surprise import SVD
from surprise import Reader, Dataset
from surprise.model_selection.search import GridSearchCV 
from surprise import KNNBasic


### Read dataset
Clean data and use products columns for training model

In [0]:
#train = pd.read_csv("/content/drive/My Drive/ADA Project/train_ver2.csv")
#test = pd.read_csv("/content/drive/My Drive/ADA Project/test_ver2.csv")
df = pd.read_csv("/content/drive/My Drive/ADA Project/data_clean_v3.csv")

In [0]:
df

,Unnamed: 0,Unnamed: 0.1,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,0,2015-01-28,1375586,N,ES,H,35,0.0,6,1.0,1.0,A,S,N,KHL,N,MALAGA,1.0,87218.10,02 - PARTICULARES,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1,1,2015-01-28,1050611,N,ES,V,23,0.0,35,1.0,1.0,I,S,S,KHE,N,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2,2,2015-01-28,1050612,N,ES,V,23,0.0,35,1.0,1.0,I,S,N,KHE,N,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,3,5,2015-01-28,1050615,N,ES,H,23,0.0,35,1.0,1.0,I,S,N,KHE,N,TOLEDO,0.0,22220.04,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,4,6,2015-01-28,1050616,N,ES,H,23,0.0,35,1.0,1.0,I,S,N,KHE,N,LEON,0.0,295590.36,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8574268,8574268,13621261,2016-05-28,1166767,N,ES,V,25,0.0,33,1.0,1.0,I,S,S,KHE,N,ZARAGOZA,0.0,73134.81,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
8574269,8574269,13621262,2016-05-28,1166766,N,ES,V,25,0.0,33,1.0,1.0,I,S,N,KHE,N,ZARAGOZA,0.0,50945.25,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
8574270,8574270,13621263,2016-05-28,1166765,N,ES,V,22,0.0,33,1.0,1.0,I,S,N,KHE,N,ZARAGOZA,0.0,43912.17,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
8574271,8574271,13621264,2016-05-28,1166764,N,ES,V,23,0.0,33,1.0,1.0,I,S,N,KHE,N,"RIOJA, LA",0.0,23334.99,03 - UNIVERSITARIO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [0]:
df.shape

(8574273, 41)

In [0]:
df.dtypes

Unnamed: 0                 int64
Unnamed: 0.1               int64
fecha_dato                object
ncodpers                   int64
ind_empleado              object
pais_residencia           object
sexo                      object
age                        int64
ind_nuevo                float64
antiguedad                 int64
indrel                   float64
indrel_1mes              float64
tiprel_1mes               object
indresi                   object
indext                    object
canal_entrada             object
indfall                   object
nomprov                   object
ind_actividad_cliente    float64
renta                    float64
segmento                  object
ind_cco_fin_ult1           int64
ind_cder_fin_ult1          int64
ind_cno_fin_ult1           int64
ind_ctju_fin_ult1          int64
ind_ctma_fin_ult1          int64
ind_ctop_fin_ult1          int64
ind_ctpp_fin_ult1          int64
ind_dela_fin_ult1          int64
ind_ecue_fin_ult1          int64
ind_fond_f

In [0]:
df['fecha_dato'].unique()

array(['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28',
       '2015-05-28', '2015-06-28', '2015-07-28', '2015-08-28',
       '2015-09-28', '2015-10-28', '2015-11-28', '2015-12-28',
       '2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28',
       '2016-05-28'], dtype=object)

In [0]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']

In [0]:
df.rename(columns={"Customer code":'ID', 'ncodpers':"Customer code", 'ind_empleado':"Employee index",
       'pais_residencia':"Customer's Country residence", 'sexo':"Customer's sex", 'age':"Age", 'ind_nuevo':"New customer Index",
       'antiguedad':"Customer seniority", 'indrel':"First/Primary", 'indrel_1mes':"Customer type at the beginning of the month", 'tiprel_1mes':"Customer relation type at the beginning of the month", 'indresi':"Residence index",
       'indext':"Foreigner index", 'canal_entrada':"channel", 'indfall':"Deceased index", 'tipodom':"Addres type", 'cod_prov':"Province code", 'nomprov':"Province name",
       'ind_actividad_cliente':"Activity index", 'renta':"Gross income of the household", 'segmento':"segmentation", 'ind_ahor_fin_ult1':"Saving Account",
       'ind_aval_fin_ult1':"Guarantees", 'ind_cco_fin_ult1':"Current Accounts", 'ind_cder_fin_ult1':"Derivada Account",
       'ind_cno_fin_ult1':"Payroll Account", 'ind_ctju_fin_ult1':"Junior Account", 'ind_ctma_fin_ult1':"Más particular Account",
       'ind_ctop_fin_ult1':"particular Account", 'ind_ctpp_fin_ult1':"particular Plus Account", 'ind_deco_fin_ult1':"Short-term deposits",
       'ind_deme_fin_ult1':"Medium-term deposits", 'ind_dela_fin_ult1':"Long-term deposits", 'ind_ecue_fin_ult1':"e-account",
       'ind_fond_fin_ult1':"Funds", 'ind_hip_fin_ult1':"Mortgage", 'ind_plan_fin_ult1':"Pensions",
       'ind_pres_fin_ult1':"Loans", 'ind_reca_fin_ult1':"Taxes", 'ind_tjcr_fin_ult1':"Credit Card",
       'ind_valo_fin_ult1':"Securities", 'ind_viv_fin_ult1':"Home Account", 'ind_nomina_ult1':"Payroll",
       'ind_nom_pens_ult1':"Nom_Pensions", 'ind_recibo_ult1':"Direct Debit"}, 
                 inplace=True)

In [0]:
df.columns

Index(['fecha_dato', 'Customer code', 'Employee index',
       'Customer's Country residence', 'Customer's sex', 'Age',
       'New customer Index', 'Customer seniority', 'First/Primary',
       'Customer type at the beginning of the month',
       'Customer relation type at the beginning of the month',
       'Residence index', 'Foreigner index', 'channel', 'Deceased index',
       'Province name', 'Activity index', 'Gross income of the household',
       'segmentation', 'Current Accounts', 'Derivada Account',
       'Payroll Account', 'Junior Account', 'Más particular Account',
       'particular Account', 'particular Plus Account', 'Long-term deposits',
       'e-account', 'Funds', 'Mortgage', 'Pensions', 'Loans', 'Taxes',
       'Credit Card', 'Securities', 'Home Account', 'Payroll', 'Nom_Pensions',
       'Direct Debit'],
      dtype='object')

In [0]:
customers = df[['fecha_dato','Customer code']]
customers.head()

,fecha_dato,Customer code
0,2015-01-28,1375586
1,2015-01-28,1050611
2,2015-01-28,1050612
3,2015-01-28,1050615
4,2015-01-28,1050616


In [0]:
train = df.loc[:, 'Current Accounts':]
train = pd.concat([customers, train], axis=1)
train['Payroll'] = train['Payroll'].astype(np.int64) 
train['Nom_Pensions'] = train['Nom_Pensions'].astype(np.int64) 
train.dtypes

fecha_dato                 object
Customer code               int64
Current Accounts            int64
Derivada Account            int64
Payroll Account             int64
Junior Account              int64
Más particular Account      int64
particular Account          int64
particular Plus Account     int64
Long-term deposits          int64
e-account                   int64
Funds                       int64
Mortgage                    int64
Pensions                    int64
Loans                       int64
Taxes                       int64
Credit Card                 int64
Securities                  int64
Home Account                int64
Payroll                     int64
Nom_Pensions                int64
Direct Debit                int64
dtype: object

## Select Monthly data
Select data by month to build model

In [0]:
def training_data(train,month):
    train_month = train[train['fecha_dato'] == month]
    train_month['Payroll'] = train_month['Payroll'].astype(np.int64) 
    train_month['Pensions'] = train_month['Pensions'].astype(np.int64) 
    cols = list(train_month.columns)[2:]
    cols_all = list(train_month.columns)
    train_month = train_month[cols_all].replace({ 0.0:np.nan})
    train_month['Customer code'] = train_month['Customer code'].astype(np.int64)
    data = pd.melt(train_month.set_index('Customer code')[cols].apply(pd.Series).reset_index(), 
             id_vars=['Customer code'],
             value_name='products')\
    .groupby(['Customer code', 'variable']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'})
    data = data[data['purchase_count'] > 0]
    df_matrix = pd.pivot_table(data, values='purchase_count', index='Customer code', columns='variable')
    d = df_matrix.reset_index() 
    d.index.names = ['scaled_purchase_freq'] 
    data_norm = pd.melt(d, id_vars=['Customer code'], value_name='scaled_purchase_freq')
    return data_norm 

In [0]:
def data_process(data):
  data['v_n'] = data.variable.astype("category").cat.codes
  data = data.fillna(0)
  return data

In [0]:
def build_train_and_test(data1, data2):
  reader = Reader(rating_scale=(0,1))
  data_new = Dataset.load_from_df(data1[["Customer code","v_n","scaled_purchase_freq"]], reader)
  trainset = data_new.build_full_trainset()
  testset = data2[["Customer code","v_n","scaled_purchase_freq"]]
  return trainset, testset

In [0]:
dataset = []
for i in df['fecha_dato'].unique():
  dataset.append(data_process(training_data(train, i)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
dataset[0]

,Customer code,variable,scaled_purchase_freq,v_n
0,15889,Credit Card,1.0,0
1,15890,Credit Card,1.0,0
2,15892,Credit Card,1.0,0
3,15893,Credit Card,0.0,0
4,15894,Credit Card,1.0,0
...,...,...,...,...
9854215,1379053,particular Plus Account,0.0,19
9854216,1379070,particular Plus Account,0.0,19
9854217,1379078,particular Plus Account,0.0,19
9854218,1379126,particular Plus Account,0.0,19


In [0]:
#trainset = []
#testset = []
#for i in range(len(dataset)-1):
#  trainset.append(build_train_and_test(dataset[i], dataset[i+1])[0])
#  testset.append(build_train_and_test(dataset[i], dataset[i+1])[1])

In [0]:
trainset, testset = build_train_and_test(dataset[0], dataset[1])

## Method: Matrix Factorization

### Without regularization

In [0]:
#model = []
#for i in range(len(dataset)-2):
#  algo = SVD()
#  model.append(algo.fit(trainset[i]))

In [0]:
algo = SVD()
algo.fit(trainset)

In [0]:
#predict = []
#test = []
#for i in range(len(dataset)-2):
#  for index, row in testset[i].iterrows():
#    est = model[i].predict(row["Customer code"], row["v_n"]).est
#    test.append((row['Customer code'], row["v_n"], est))
#  test = pd.DataFrame(test, columns=['Customer code', 'v_n', 'est'])
#  predict.append(test)

In [0]:
test = []
for index, row in testset.iterrows():
  est = algo.predict(row["Customer code"], row["v_n"]).est
  test.append((row['Customer code'], row["v_n"], est))
test = pd.DataFrame(test, columns=['Customer code', 'v_n', 'est'])

In [0]:
pred = test[test.est>0.2]

In [0]:
pred['recommend'] = 1

In [0]:
pred

,Customer code,v_n,est,recommend
1,15890.0,0.0,0.305780,1
2,15892.0,0.0,0.259047,1
4,15894.0,0.0,0.385751,1
5,15895.0,0.0,0.343357,1
6,15897.0,0.0,0.429926,1
...,...,...,...,...
9850331,1367514.0,19.0,0.162387,1
9850423,1367750.0,19.0,0.163426,1
9851203,1370777.0,19.0,0.208515,1
9851523,1371827.0,19.0,0.166396,1


### With regularization


In [0]:
trainset, testset = build_train_and_test(dataset[0], dataset[1])

In [0]:
algo1 = SVDpp(lr_all= 0.002, n_epochs= 15, reg_all=0.6)
algo1.fit(trainset)

In [0]:
test1 = []
for index, row in testset.iterrows():
  est = algo1.predict(row["Customer code"], row["v_n"]).est
  test1.append((row['Customer code'], row["v_n"], est))
test1 = pd.DataFrame(test1, columns=['Customer code', 'v_n', 'est'])

In [0]:
pred1 = test1[test1.est>0.2]

In [0]:
pred1['recommend'] = 1

### Evaluation

In [0]:
def evaluate(train, prediction, groundtruth):
  Actual = pd.merge(train, groundtruth, 'outer', on = ['Customer code', 'v_n'])
  All = pd.merge(Actual, prediction, 'outer', on = ['Customer code', 'v_n'])
  Precision_All = pd.merge(Actual, prediction, 'inner', on = ['Customer code', 'v_n'])
  precision = np.sum(Precision_All[Precision_All.scaled_purchase_freq_x == 0].scaled_purchase_freq_y == 1)/np.sum(Precision_All.scaled_purchase_freq_x == 0)
  recall = np.sum(All[(All.scaled_purchase_freq_x == 0) & (All.scaled_purchase_freq_y == 1)].recommend)/np.sum((All.scaled_purchase_freq_x == 0) & (All.scaled_purchase_freq_y == 1))
  return precision, recall


final result

In [0]:
evaluate(dataset[-2], pred1, testset)

(0.01630998770159126, 0.3993175635173145)

last month training

In [0]:
evaluate(dataset[-3], pred1, testset)

(0.016152187498240655, 0.4403007980356047)

first month training

In [0]:
evaluate(dataset[0], pred1, testset)

(0.00909394053688157, 0.571165235125701)